### Read in and save output EQT results

In [ ]:
import numpy as np
import pandas as pd
import shutil
import os
from zipfile import ZipFile
import glob
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import obspy
import glob2 as glob
from obspy.clients.fdsn import Client
from collections import defaultdict
import pyarrow

In [ ]:
base_dirs = ['/home/zkrauss/lynx/end_2017_enwf1','/home/zkrauss/lynx/end_2017_enwfkemo2','/home/zkrauss/lynx/end_2017_kemf1','/home/zkrauss/lynx/end_2017_kemf2','/home/zkrauss/lynx/end_2017_kemo1','/home/zkrauss/lynx/end_2017_nchr1','/home/zkrauss/lynx/end_2017_nchr2']
res_files = []
for d in base_dirs:
    res_files.extend(glob.glob(d + '/**/X*results.csv'))

In [ ]:
eqt = pd.concat([pd.read_csv(f,engine='pyarrow') for f in res_files])

In [ ]:
# Since P and S waves are saved together, need to split them to compare--
eqt_p = eqt.drop(columns=['s_arrival_time','s_probability','s_uncertainty','s_snr'])
eqt_p=eqt_p.rename(columns={'p_arrival_time': "arrival_time", 'p_probability': "probability",'p_uncertainty':"uncertainty",'p_snr':"snr"})
eqt_p['arrival_time']=pd.to_datetime(eqt_p['arrival_time'],infer_datetime_format=True)
eqt_p = eqt_p.dropna(subset=['arrival_time'])
eqt_s = eqt.drop(columns=['p_arrival_time','p_probability','p_uncertainty','p_snr'])
eqt_s=eqt_s.rename(columns={'s_arrival_time': "arrival_time", 's_probability': "probability",'s_uncertainty':"uncertainty",'s_snr':"snr"})
eqt_s['arrival_time']=pd.to_datetime(eqt_s['arrival_time'],infer_datetime_format=True)
eqt_s = eqt_s.dropna(subset=['arrival_time'])
eqt_p['phase']=['P']*len(eqt_p)
eqt_s['phase']=['S']*len(eqt_s)
eqt = pd.concat([eqt_p,eqt_s])
eqt.head()

In [ ]:
eqt.to_parquet(path='eqtresults_2017.parquet',version='2.6')